In [62]:
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyfit

In [58]:
xls = pd.ExcelFile('leap_rig_opto.xlsx')

In [59]:
def create_stim(excelSheet, desiredPower, useCoverslip=True):
    """returns the voltage required to produced the desiredPower"""

    if useCoverslip:
        # uses measurements taken underneath coverslip (decreased power by < 3mW/cm2)
        red = pd.read_excel(excelSheet, 'red_coverslip')
        green = pd.read_excel(excelSheet, 'green_coverslip')
    else:
        red = pd.read_excel(excelSheet, 'red_linearity')
        green = pd.read_excel(excelSheet, 'green_linearity')

    RedPowerDict={}
    GreenPowerDict={}
    for rig in red['Rig'].unique():
        redrig = red[red['Rig']==rig][1:] #doesn't include the reading from 0V
        redvoltages = np.array(redrig['Voltage'])
        redpower = np.array(redrig['Power Reading (mW/cm2)'])
        idx = ~np.isnan(redpower) #excludes nan values
        redvoltages = redvoltages[idx]
        redpower = redpower[idx]

        greenrig = green[green['Rig']==rig][1:] #doesn't include the reading from 0V
        greenvoltages = np.array(greenrig['Voltage'])
        greenpower = np.array(greenrig['Power Reading (mW/cm2)'])
        idx = ~np.isnan(greenpower) #excludes nan values
        greenvoltages = greenvoltages[idx]
        greenpower = greenpower[idx]

        rb,rm = polyfit(redvoltages, redpower,1)
        gb,gm = polyfit(greenvoltages, greenpower,1)
        redV = (desiredPower-rb)/rm
        greenV = (desiredPower-gb)/gm

        RedPowerDict[rig]=redV
        GreenPowerDict[rig]=greenV

    return RedPowerDict, GreenPowerDict

        

## get voltages for a given power

In [60]:
red, green = create_stim(xls, 10)

In [61]:
red

{1.1: 1.7819603730087625,
 1.2: 1.7644961008239148,
 2.1: 1.8120507834902528,
 2.2: 1.830408589772224,
 3.1: 1.7604514212183595,
 3.2: 1.7597216320322564}

## create a stimulus file using python

In [84]:
from scipy.io import savemat
import numpy as np

In [95]:
voltage = 1.2345

fs = 10000 # samplerate
stimLength = 5 # seconds
daqStimLength = stimLength*fs

stim = {'stim': np.array([np.ones(daqStimLength)*voltage]).T}

# if you want flashing on and off:
on = np.ones(daqStimLength)*voltage
off = np.zeros(daqStimLength)
stimulus = np.append(on, off)

stim = {'stim': np.array([stimulus]).T}

In [96]:
savemat('example_stim.mat', stim)